In [1]:
# Import dependencies

from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
# Setup splinter for the whole project
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/kennethandersen/.wdm/drivers/chromedriver/mac64/91.0.4472.19/chromedriver] found in cache


In [216]:
##########################################################################
#                            Scrape Article
##########################################################################

# Setup url and launch browser
article_url = 'https://redplanetscience.com'             
browser.visit(article_url)

In [217]:
# Use BeautifulSoup to read the url and extract the article Header and Teaser and assign to variables

soup = BeautifulSoup(browser.html, 'html.parser')

article_headers = soup.find_all('div', class_='content_title')
first_article_header = headers[0].text

article_teasers = soup.find_all('div', class_='article_teaser_body')
first_article_teaser = article_teasers[0].text

print(first_article_header)
print(first_article_teaser)

Virginia Middle School Student Earns Honor of Naming NASA's Next Mars Rover
NASA chose a seventh-grader from Virginia as winner of the agency's "Name the Rover" essay contest. Alexander Mather's entry for "Perseverance" was voted tops among 28,000 entries. 


In [218]:
##########################################################################
#                         Scrape Featured Image
##########################################################################

# Setup url and launch browser
image_url = 'https://spaceimages-mars.com/'
browser.visit(image_url)

In [219]:
# Use BeautifulSoup to read the url and extract the featured image url

soup = BeautifulSoup(browser.html, 'html.parser')

featured_images = soup.find_all('img', class_='headerimage fade-in')

featured_image_url = f"{image_url}{featured_images[0]['src']}"
print(featured_image_url)

https://spaceimages-mars.com/image/featured/mars1.jpg


In [4]:
##########################################################################
#                         Scrape Fact Table
##########################################################################

# Setup url and launch browser
mars_facts_table_url = 'https://galaxyfacts-mars.com/'
browser.visit(mars_facts_table_url)

In [5]:
# Use Pandas to read the url and extract the table

mars_facts_df = pd.read_html(mars_facts_table_url)[0]
mars_facts_df.columns = ['Mars - Earth Comparison', 'Mars', 'Earth']  # Clean up column names
mars_facts_df = mars_facts_df.iloc[1:]                                # Get rid of first row
mars_facts_df.set_index('Mars - Earth Comparison', inplace=True)      # Replace numbered index
mars_facts_df

,Mars,Earth
Mars - Earth Comparison,,
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [7]:
# Export the table to html file

mars_facts_df.to_html()


'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>\n</table>'

In [223]:
##########################################################################
#                      Scrape Hemisphere Images
##########################################################################

# Setup url and launch browser
hemispheres_url = 'https://marshemispheres.com/'
browser.visit(hemispheres_url)

# Use BeautifulSoup to read the url and extract the description section 
#for each of the 4 "Products" (Hemispheres)
soup = BeautifulSoup(browser.html, 'html.parser')
products = soup.find_all('div', class_= 'description')

In [226]:
# Iterate over the text for each "Product" (Hemisphere) and extract the title and url for the 
#high res tiff file.

hemisphere_images = []                            # Set up list for results, as instructed

for product in products:
    
    temp_dict = {}                                # Set up temporary dictionary for results
    
    # Get image title
    hemisphere_image_title = product.find('h3').text      # Extract title from the <h3> tags
    temp_dict['image_title'] = hemisphere_image_title     # Add Image Title to dictionary
    
    
    # Get url for high resolution tif file
    hemisphere_url = product.find_all('a', class_='itemLink product-item')[0]['href'] # Get link for product page
    browser.visit(f"{hemisphere_images_url}{hemisphere_url}")                         # Send browser to product page
    soup = BeautifulSoup(browser.html, 'html.parser')                                 # Parse product page
    
    # Drill down into text to get the .tif url
    hemisphere_image_url = soup.find('div', {'class' :'downloads'}).find('ul').find_all('a')[1]['href']                               
    hemisphere_image_url_complete = f"{hemisphere_images_url}{hemisphere_image_url}"  # Piece together the full url
    temp_dict['image_url'] = hemisphere_image_url_complete                            # Add url to dictionary
    
    # Append dictionary to list
    hemisphere_images.append(temp_dict)
    
hemisphere_images  

[{'image_title': 'Cerberus Hemisphere Enhanced',
  'image_url': 'https://marshemispheres.com/images/cerberus_enhanced.tif'},
 {'image_title': 'Schiaparelli Hemisphere Enhanced',
  'image_url': 'https://marshemispheres.com/images/schiaparelli_enhanced.tif'},
 {'image_title': 'Syrtis Major Hemisphere Enhanced',
  'image_url': 'https://marshemispheres.com/images/syrtis_major_enhanced.tif'},
 {'image_title': 'Valles Marineris Hemisphere Enhanced',
  'image_url': 'https://marshemispheres.com/images/valles_marineris_enhanced.tif'}]

In [227]:
browser.quit()